# Flocks, Herds, and Traffic Jams

Code examples from [Think Complexity, 2nd edition](https://thinkcomplex.com).

Copyright 2016 Allen Downey, [MIT License](http://opensource.org/licenses/MIT)

In [1]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import thinkplot
from thinkstats2 import RandomSeed

from Cell2D import Cell2DViewer

from matplotlib import rc
rc('animation', html='jshtml')

Here's the `Driver` class from Chapter 10.

In [2]:
class Driver:
    
    def __init__(self, loc, speed=4):
        """Initialize the attributes.
        
        loc: position on track, in miles
        speed: speed in miles per hour
        """
        self.start = loc
        self.loc = loc
        self.speed = speed
        
    def choose_acceleration(self, dist):
        """Chooses acceleration based on distance.
        
        dist: distance from the car in front
        
        returns: acceleration
        """
        return 1
        
    def set_odometer(self):
        self.start = self.loc
        
    def read_odometer(self):
        return self.loc - self.start

And the Highway.

In [3]:
class Highway:
    
    max_acc = 1
    min_acc = -10
    speed_limit = 40
    
    def __init__(self, n=10, length=1000, eps=0, constructor=Driver):
        """Initializes the attributes.
        
        n: number of drivers
        length: length of the track
        eps: variability in speed
        constructor: function used to instantiate drivers
        """
        self.length = length
        self.eps = eps
        self.crashes = 0

        # create the drivers
        locs = np.linspace(0, length, n, endpoint=False)
        self.drivers = [constructor(loc) for loc in locs]
        
        # and link them up
        for i in range(n):
            j = (i+1) % n
            self.drivers[i].next = self.drivers[j]
            
    def step(self):
        """Performs one time step."""
        for driver in self.drivers:
            self.move(driver)
            
    def move(self, driver):
        """Updates `driver`.
        
        driver: Driver object
        """
        # get the distance to the next driver
        dist = self.distance(driver)

        # let the driver choose acceleration
        acc = driver.choose_acceleration(dist)
        acc = min(acc, self.max_acc)
        acc = max(acc, self.min_acc)
        speed = driver.speed + acc
            
        # add random noise to speed
        speed *= np.random.uniform(1-self.eps, 1+self.eps)
        
        # keep it nonnegative and under the speed limit
        speed = max(speed, 0)
        speed = min(speed, self.speed_limit)
        
        # if current speed would collide with next driver, stop
        if speed > dist:
            speed = 0
            self.crashes += 1
            
        # update speed and loc
        driver.speed = speed
        driver.loc += speed
            
    def distance(self, driver):
        """Distance from `driver` to next driver.
        
        driver: Driver object
        """
        dist = driver.next.loc - driver.loc
        # fix wraparound
        if dist < 0:
            dist += self.length
        return dist
    
    def set_odometers(self):
        return [driver.set_odometer()
                for driver in self.drivers] 
    
    def read_odometers(self):
        return np.mean([driver.read_odometer()
                        for driver in self.drivers])

The `HighwayViewer` is based on the Cell2DViewer (although it does not draw a grid).

In [4]:
class HighwayViewer(Cell2DViewer):
    """Generates visualization and animation of SugarScape."""
    
    def draw(self, grid=False):
        """Draws the array and any other elements.
        
        grid: boolean, whether to draw grid lines
        """
        self.draw_drivers()
        
        plt.axis('off')
        plt.axis('equal')
        plt.xlim([-1.05, 1.05])
        plt.ylim([-1.05, 1.05])

    def draw_drivers(self):
        """Plots the drivers.
        """
        drivers = self.viewee.drivers
        xs, ys = self.get_coords(drivers)
        self.points = plt.plot(xs, ys, 'bs', markersize=10, alpha=0.7)[0]
        
        stopped = [driver for driver in self.viewee.drivers 
                  if driver.speed==0]
        xs, ys = self.get_coords(stopped)
        xs *= 0.9
        ys *= 0.9
        self.stopped = plt.plot(xs, ys, 'r^', markersize=12, alpha=0.7)[0]
        
    def animate_func(self, i):
        """Draws one frame of the animation."""
        if i > 0:
            self.step()
        drivers = self.viewee.drivers
        xs, ys = self.get_coords(drivers)
        self.points.set_data(np.array([xs, ys]))
        
        stopped = [driver for driver in self.viewee.drivers 
                  if driver.speed==0]
        xs, ys = self.get_coords(stopped)
        self.stopped.set_data(np.array([xs, ys]) * 0.9)
        return self.points,
    
    def get_coords(self, drivers):
        """Gets the coordinates of the drivers.
        
        Transforms from (row, col) to (x, y).
        
        returns: tuple of sequences, (xs, ys)
        """
        scale = 2 * np.pi / self.viewee.length
        
        locs = np.array([driver.loc for driver in drivers])
        xs = np.cos(locs * scale)
        ys = np.sin(locs * scale)
        return xs, ys

Make the animation:

In [5]:
highway = Highway(30, eps=0.02)
viewer = HighwayViewer(highway)
anim = viewer.animate(frames=600, interval=200)

In [6]:
anim

And generate the figure:

In [7]:
RandomSeed(22)

highway = Highway(30, eps=0.02)
viewer = HighwayViewer(highway)

thinkplot.preplot(cols=3)
for i in range(16):
    highway.step()
viewer.draw()

thinkplot.subplot(2)
for i in range(1):
    highway.step()
viewer.draw()

thinkplot.subplot(3)
for i in range(1):
    highway.step()
viewer.draw()

thinkplot.tight_layout()
thinkplot.save('chap10-1')

Explore the relationship between `eps` and average speed.

In [8]:
def run_simulation(eps, constructor=Driver, iters=100):
    res = []
    for n in range(5, 100, 5):
        highway = Highway(n, eps=eps, constructor=constructor)
        for i in range(iters):
            highway.step()

        highway.set_odometers()
        for i in range(iters):
            highway.step()

        res.append((n, highway.read_odometers() / iters))
    
    return np.transpose(res)

Generate the figure:

In [9]:
RandomSeed(20)

for eps in [0.0, 0.001, 0.01]:
    xs, ys = run_simulation(eps)
    thinkplot.plot(xs, ys, label='eps=%g' % eps)
    
thinkplot.config(xlabel='Number of cars',
                 ylabel='Average speed',
                 xlim=[0, 100], ylim=[0, 42])

thinkplot.save('chap10-2')

**Exercise:** In the traffic jam simulation, define a class, `BetterDriver`,
that inherits from `Driver` and overrides `choose_acceleration`.
See if you can define driving rules that do better than the basic
implementation in `Driver`.  You might try to achieve higher
average speed, or a lower number of collisions.

In [10]:
# Not actually a solution yet; just playing around.

class BetterDriver(Driver):
    
    def choose_acceleration(self, d):
        if self.speed < 20:
            return 1
        else:
            return 0

In [11]:
for constructor in [Driver, BetterDriver]:
    xs, ys = run_simulation(eps=0.0, constructor=constructor)
    thinkplot.plot(xs, ys, label=constructor.__name__)
    
thinkplot.config(xlabel='Number of cars',
                 ylabel='Average speed',
                 xlim=[0, 100], ylim=[0, 42])